In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
import os
import glob as gb
import cv2
import PIL
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from keras.models import Model

In [9]:
trainpath = '/kaggle/input/riceleafdiseasedataset/dataset/train'
testpath = '/kaggle/input/riceleafdiseasedataset/dataset/test'

# Image Processing

In [10]:
# Image Processing - Training Data
new_size = 224
train_images = []
train_labels = []
class_disease = {'BacterialBlight': 0, 'Blast': 1, 'BrownSpot': 2, 'Tungro': 3}

for i in os.listdir(trainpath):
    if i in class_disease:
        print("Entering the folder:", i)
        files = gb.glob(pathname=str(trainpath + '/' + i + '/*.jpg')) + gb.glob(pathname=str(trainpath + '/' + i + '/*.JPG'))
        print("Number of images in the folder:", len(files))
        for j in files:
            image_raw = cv2.imread(j)
            image = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)
            resize_image = cv2.resize(image, (new_size, new_size))
            train_images.append(list(resize_image))
            train_labels.append(class_disease[i])

# Image Processing - Testing Data
new_size = 224
test_images = []
test_labels = []

for i in os.listdir(testpath):
    if i in class_disease:
        print("Entering to the folder name:", i)
        files = gb.glob(pathname=str(testpath + '/' + i + '/*.jpg')) + gb.glob(pathname=str(testpath + '/' + i + '/*.JPG'))
        print("Number of images in the folder is", len(files))
        for j in files:
            image_raw = cv2.imread(j)
            image = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)
            resize_image = cv2.resize(image, (new_size, new_size))
            test_images.append(list(resize_image))
            test_labels.append(class_disease[i])

def list_to_array_train(train_images, train_labels):
    return np.array(train_images), np.array(train_labels)

X_train, y_train = list_to_array_train(train_images, train_labels)

def list_to_array_test(test_images, test_labels):
    return np.array(test_images), np.array(test_labels)

X_test, y_test = list_to_array_test(test_images, test_labels)

print(X_train.shape)
print("*" * 20)
print(y_train.shape)
print("*" * 20)
print(X_test.shape)
print(y_test.shape)

def keras_to_categorical(y_train, y_test):
    return to_categorical(y_train), to_categorical(y_test)

y_train1 = y_train
y_test1 = y_test
y_train, y_test = keras_to_categorical(y_train, y_test)

y_train1.shape, y_test1.shape

def convert_one_hot_to_categorical(one_hot_labels):
    return np.argmax(one_hot_labels, axis=1)

gpus = tf.config.list_physical_devices('GPU')
print(f"Num GPUs Available: {len(gpus)}")

if len(gpus) < 2:
    print("Not enough GPUs available, ensure your environment is configured correctly")
else:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    strategy = tf.distribute.MirroredStrategy()

Entering the folder: Tungro
Number of images in the folder: 1108
Entering the folder: BacterialBlight
Number of images in the folder: 1384
Entering the folder: Blast
Number of images in the folder: 1240
Entering the folder: BrownSpot
Number of images in the folder: 1400
Entering to the folder name: Tungro
Number of images in the folder is 200
Entering to the folder name: BacterialBlight
Number of images in the folder is 200
Entering to the folder name: Blast
Number of images in the folder is 200
Entering to the folder name: BrownSpot
Number of images in the folder is 200
(5132, 224, 224, 3)
********************
(5132,)
********************
(800, 224, 224, 3)
(800,)
Num GPUs Available: 2


In [11]:
# Function to convert one-hot encoded labels to categorical labels
def convert_one_hot_to_categorical(one_hot_labels):
    return np.argmax(one_hot_labels, axis=1)

In [14]:
# Deep Feature Extraction - ResNet50
def model_resnet50():
    Resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in Resnet_model.layers:
        layer.trainable = False
    feature = GlobalAveragePooling2D()(Resnet_model.output)
    output = Model(inputs=Resnet_model.input, outputs=feature)
    return output

# Deep Feature Extraction - InceptionV3
def model_inceptionv3():
    Inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in Inception_model.layers:
        layer.trainable = False
    feature = GlobalAveragePooling2D()(Inception_model.output)
    output = Model(inputs=Inception_model.input, outputs=feature)
    return output

# ResNet50 + InceptionV3 + Random Forest Classifier

In [15]:
from sklearn.ensemble import RandomForestClassifier

def train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():  
        model_FE_resnet = model_resnet50()
        model_FE_resnet.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_50 = model_FE_resnet.predict(X_train)
        test_feature_50 = model_FE_resnet.predict(X_test)
        
        model_FE_inception = model_inceptionv3()
        model_FE_inception.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_v3 = model_FE_inception.predict(X_train)
        test_feature_v3 = model_FE_inception.predict(X_test)
        
        final_train = np.hstack((train_feature_50, train_feature_v3))
        final_test = np.hstack((test_feature_50, test_feature_v3))

        rf = RandomForestClassifier()
        rf = rf.fit(final_train, y_train)
        test_pred = rf.predict(final_test)

        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Calculate average metrics
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

# Print average metrics
print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'InceptionV3+ResNet50+RF.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 89ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 22s 88ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step
Run 1 - Accuracy: 0.96500000, Recall: 0.96500000, Precision: 0.99744898, F1 Score: 0.98047069, AUC: 0.98208333
161/161 ━━━━━━━━━━━━━━━━━━━━ 22s 93ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 77ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
Run 2 - Accuracy: 0.96125000, Recall: 0.96125000, Precision: 0.99747475, F1 Score: 0.97858779, AUC: 0.98020833
161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 91ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 76ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
Run 3 - Accuracy: 0.97250000, Recall: 0.97250000, Precision: 0.99752475, F1 Score: 0.98460937, AUC: 0.98583333
161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 91ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 75

# ResNet50 + InceptionV3 + Decision Tree Classifier

In [16]:
from sklearn.tree import DecisionTreeClassifier

def train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():  
        model_FE_resnet = model_resnet50()
        model_FE_resnet.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_50 = model_FE_resnet.predict(X_train)
        test_feature_50 = model_FE_resnet.predict(X_test)
        
        model_FE_inception = model_inceptionv3()
        model_FE_inception.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_v3 = model_FE_inception.predict(X_train)
        test_feature_v3 = model_FE_inception.predict(X_test)
        
        final_train = np.hstack((train_feature_50, train_feature_v3))
        final_test = np.hstack((test_feature_50, test_feature_v3))

        # Define and train Decision Tree Classifier
        dt = DecisionTreeClassifier()
        dt.fit(final_train, y_train)
        test_pred = dt.predict(final_test)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'InceptionV3+ResNet50+DTC.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 92ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 77ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
Run 1 - Accuracy: 0.91750000, Recall: 0.91750000, Precision: 0.91720968, F1 Score: 0.91711257, AUC: 0.94500000
161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 91ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 76ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
Run 2 - Accuracy: 0.90625000, Recall: 0.90625000, Precision: 0.90591145, F1 Score: 0.90570742, AUC: 0.93750000
161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 91ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 82ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step
Run 3 - Accuracy: 0.91125000, Recall: 0.91125000, Precision: 0.91107838, F1 Score: 0.91057803, AUC: 0.94083333
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 93ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 78ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
Run

# ResNet50 + InceptionV3 + KNN Classifier

In [17]:
from sklearn.neighbors import KNeighborsClassifier

def train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test, num):
    with strategy.scope():  
        model_FE_resnet = model_resnet50()
        model_FE_resnet.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_50 = model_FE_resnet.predict(X_train)
        test_feature_50 = model_FE_resnet.predict(X_test)
        
        model_FE_inception = model_inceptionv3()
        model_FE_inception.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_v3 = model_FE_inception.predict(X_train)
        test_feature_v3 = model_FE_inception.predict(X_test)
        
        final_train = np.hstack((train_feature_50, train_feature_v3))
        final_test = np.hstack((test_feature_50, test_feature_v3))

        # Define and train KNN Classifier
        knn = KNeighborsClassifier(n_neighbors=num)
        knn.fit(final_train, y_train)
        test_pred = knn.predict(final_test)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}
num = 5

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test, num)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")
    num = num+1

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'InceptionV3+ResNet50+KNN.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


161/161 ━━━━━━━━━━━━━━━━━━━━ 20s 96ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 79ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step
Run 1 - Accuracy: 0.83125000, Recall: 0.83125000, Precision: 0.85306691, F1 Score: 0.83679927, AUC: 0.89229167
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 93ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 77ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step
Run 2 - Accuracy: 0.79000000, Recall: 0.79000000, Precision: 0.87296616, F1 Score: 0.81976032, AUC: 0.87666667
161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 92ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 75ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
Run 3 - Accuracy: 0.80625000, Recall: 0.80625000, Precision: 0.83545402, F1 Score: 0.81214096, AUC: 0.87791667
161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 92ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 23s 82ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step
Run

# ResNet50 + InceptionV3 + SVM Classifier

In [18]:
from sklearn.svm import SVC

def train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():  
        y_train_cat = convert_one_hot_to_categorical(y_train)
        y_test_cat = convert_one_hot_to_categorical(y_test)
        
        model_FE_resnet = model_resnet50()
        model_FE_resnet.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_50 = model_FE_resnet.predict(X_train)
        test_feature_50 = model_FE_resnet.predict(X_test)
        
        model_FE_inception = model_inceptionv3()
        model_FE_inception.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_v3 = model_FE_inception.predict(X_train)
        test_feature_v3 = model_FE_inception.predict(X_test)
        
        final_train = np.hstack((train_feature_50, train_feature_v3))
        final_test = np.hstack((test_feature_50, test_feature_v3))

        # Define and train SVM Classifier
        svm = SVC(probability=True)
        svm.fit(final_train, y_train_cat)
        test_pred = svm.predict(final_test)
        test_pred_proba = svm.predict_proba(final_test)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test_cat, test_pred)
        recall = recall_score(y_test_cat, test_pred, average='weighted')
        precision = precision_score(y_test_cat, test_pred, average='weighted')
        f1 = f1_score(y_test_cat, test_pred, average='weighted')
        auc = roc_auc_score(y_test_cat, test_pred_proba, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'InceptionV3+ResNet50+SVM.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 92ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 76ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step
Run 1 - Accuracy: 0.83750000, Recall: 0.83750000, Precision: 0.84065009, F1 Score: 0.83345066, AUC: 0.96321458
161/161 ━━━━━━━━━━━━━━━━━━━━ 23s 123ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 81ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step
Run 2 - Accuracy: 0.83750000, Recall: 0.83750000, Precision: 0.84065009, F1 Score: 0.83345066, AUC: 0.96323750
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 92ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 76ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step
Run 3 - Accuracy: 0.83750000, Recall: 0.83750000, Precision: 0.84065009, F1 Score: 0.83345066, AUC: 0.96324583
161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 89ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 74ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step
Ru

# ResNet50 + InceptionV3 + XGBoost Classifier

In [19]:
from xgboost import XGBClassifier

def train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():  
        y_train_cat = convert_one_hot_to_categorical(y_train)
        y_test_cat = convert_one_hot_to_categorical(y_test)
        
        model_FE_resnet = model_resnet50()
        model_FE_resnet.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_50 = model_FE_resnet.predict(X_train)
        test_feature_50 = model_FE_resnet.predict(X_test)
        
        model_FE_inception = model_inceptionv3()
        model_FE_inception.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_v3 = model_FE_inception.predict(X_train)
        test_feature_v3 = model_FE_inception.predict(X_test)
        
        final_train = np.hstack((train_feature_50, train_feature_v3))
        final_test = np.hstack((test_feature_50, test_feature_v3))

        # Define and train XGB Classifier
        xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
        xgb.fit(final_train, y_train_cat)
        test_pred = xgb.predict(final_test)
        test_pred_proba = xgb.predict_proba(final_test)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test_cat, test_pred)
        recall = recall_score(y_test_cat, test_pred, average='weighted')
        precision = precision_score(y_test_cat, test_pred, average='weighted')
        f1 = f1_score(y_test_cat, test_pred, average='weighted')
        auc = roc_auc_score(y_test_cat, test_pred_proba, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'InceptionV3+ResNet50+XGB.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 90ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 75ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
Run 1 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 0.99998750
161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 89ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 75ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
Run 2 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 0.99998750
161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 90ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 24s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
Run 3 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 0.99998750
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 93ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 77ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step
Run